In [265]:
import numpy as np
from tqdm import tqdm
from time import sleep
import operator

In [266]:
class NN():

    #creates a 3 layer NN with 2 input neuron, n hidden neuron and 1 ouput neuron
    def __init__(self, hiddenLayer_neuron = 2, pop_size = 2):

        self.data = np.array([[0,0],
                              [0,1],
                              [1,0],
                              [1,1]])
        self.target = np.array([[0],[1],[1],[0]])

        self.hl_neuron = hiddenLayer_neuron

        self.pop_size = pop_size
        
        self.w_hidden = [np.random.uniform(size=(2,self.hl_neuron)) for _ in range(pop_size)]
        self.w_out = [np.random.uniform(size=(self.hl_neuron,1)) for _ in range(pop_size)]

        self.b_hidden = [np.random.uniform(size=(1,self.hl_neuron)) for _ in range(pop_size)]
        self.b_out = [np.random.uniform(size=(1,1)) for _ in range(pop_size)]

        self.z0s = []
        self.z1s = []

        print("Init: ")
        self.print_weights()

    #ReLU function
    def ReLU(self,x):
        return x * (x > 0) 
    
    #binary cross entropy loss function
    def bcel(self, y, y_hat):
        return -(y*np.log(y_hat) + (1-y) * np.log(1-y_hat))

    #sigmoid function
    def sigmoid(self,x):
        return 1/(1 + np.exp(-x))
    
    #Execute the NN with input x
    def execute(self,x):
        result = self.sigmoid(self.forward_pass(0,x)[0])
        return np.round(result[0])
    
    #forward pass
    def forward_pass(self,index,x):
        x = np.array(x)
        z0 = np.dot(x,self.w_hidden[index]) + self.b_hidden[index]
        hidden = self.ReLU(z0)
        z1 = np.dot(hidden,self.w_out[index]) + self.b_out[index]
        return z1, z0

    def neuro_train(self):
        
        
        while self.accuracy(0)[0] != 1:
            
            self.print_weights()
            c_w_out = self.crossover(self.w_out)
            c_w_hidden = self.crossover(self.w_hidden)
            c_b_out = self.crossover(self.b_out)
            c_b_hidden = self.crossover(self.b_hidden)

            self.mutation(c_w_out)
            self.mutation(c_w_hidden)
            self.mutation(c_b_out)
            self.mutation(c_b_hidden)

            self.w_out.append(c_w_out)
            self.w_hidden.append(c_w_hidden)
            self.b_out.append(c_b_out)
            self.b_hidden.append(c_b_hidden)

            f_list = self.fitness()
            self.selection(f_list)

    def fitness(self):
        f_list = []
        for i in range(len(self.w_out)-1):
            f_list.append(self.accuracy(i)[1][0])
        return f_list

    def selection(self, f_list):
        dic = dict()
    
        for i in range(len(f_list)):
            dic.update([(i,f_list[i])])
        
        sortedPop = sorted(dic.items(),key=operator.itemgetter(1),reverse=False)
        topN = sortedPop[:100]
        new_w_out = []
        new_w_hidden = []
        new_b_out = []
        new_b_hidden = []
        for index in topN:
            new_w_out.append(self.w_out[index[0]])
            new_w_hidden.append(self.w_hidden[index[0]])
            new_b_out.append(self.w_out[index[0]])
            new_b_hidden.append(self.w_hidden[index[0]])
        
        self.w_out = new_w_out
        self.w_hidden = new_w_hidden
        self.b_out = new_b_out
        self.b_hidden = new_b_hidden



    def crossover(self, weight):
        w = np.zeros(shape=(len(weight),weight[0].shape[0],weight[0].shape[1]))
        #print(weight)
        #print("len: " + str(len(weight)))
        lis = [i for i in range(len(weight))]
        np.random.shuffle(lis)
        for k in range(len(weight) // 2 - 1):
            #print("k "+ str(k))
            for i in range(weight[0].shape[0]):
                #print("i "+ str(i))
                for j in range(weight[0].shape[1]):
                    #print("j "+ str(j))
                    if np.random.randint(0,2):
                        w[lis[2*k]][i][j] = weight[lis[2*k+1]][i][j]
                        w[lis[2*k+1]][i][j] = weight[lis[2*k]][i][j]
                    else:
                        w[lis[2*k]][i][j] = weight[lis[2*k]][i][j]
                        w[lis[2*k+1]][i][j] = weight[lis[2*k+1]][i][j]
        return w

    def mutation(self, weight):
        for k in range(len(weight)):
            for i in range(weight[k].shape[0]):
                for j in range(weight[k].shape[1]):
                    weight[k][i][j] += np.random.normal(0,0.1)


    #prints the weights of the nn  
    def print_weights(self):
        print("W_hidden: " + str(self.w_hidden)) 
        #print("B_hidden: " + str(self.b_hidden)) 
        #print("W_out: " + str(self.w_out)) 
        #print("B_out: " + str(self.b_out), end="\n\n")

    #calculates the accuracy and current loss of the nn
    def accuracy(self, index):

        counter = 0
    
        for i in range(len(self.data)):
            if(self.execute(self.data[i]) == self.target[i]):
                counter += 1
        
        loss = 0
        for i in range(len(self.data)):
            loss += self.bcel(self.target[i],self.sigmoid(self.forward_pass(index,self.data[i])[0]))

        return counter / len(self.data), loss / len(self.data)
    

In [267]:
nn = NN(2, pop_size= 2)

Init: 
W_hidden: [array([[0.37935336, 0.47744609],
       [0.05298185, 0.11222395]]), array([[0.73599332, 0.83694586],
       [0.61281476, 0.09974927]])]


In [268]:
nn.neuro_train()
#nn.selection(nn.fitness())
#nn.train(epochs = -1)

W_hidden: [array([[0.37935336, 0.47744609],
       [0.05298185, 0.11222395]]), array([[0.73599332, 0.83694586],
       [0.61281476, 0.09974927]])]
W_hidden: [array([[0.37935336, 0.47744609],
       [0.05298185, 0.11222395]]), array([[0.73599332, 0.83694586],
       [0.61281476, 0.09974927]])]
W_hidden: [array([[0.37935336, 0.47744609],
       [0.05298185, 0.11222395]]), array([[0.73599332, 0.83694586],
       [0.61281476, 0.09974927]])]
W_hidden: [array([[0.37935336, 0.47744609],
       [0.05298185, 0.11222395]]), array([[0.73599332, 0.83694586],
       [0.61281476, 0.09974927]])]
W_hidden: [array([[0.37935336, 0.47744609],
       [0.05298185, 0.11222395]]), array([[0.73599332, 0.83694586],
       [0.61281476, 0.09974927]])]
W_hidden: [array([[0.37935336, 0.47744609],
       [0.05298185, 0.11222395]]), array([[0.73599332, 0.83694586],
       [0.61281476, 0.09974927]])]
W_hidden: [array([[0.37935336, 0.47744609],
       [0.05298185, 0.11222395]]), array([[0.73599332, 0.83694586],
      

KeyboardInterrupt: 